<a href="https://colab.research.google.com/github/1moi6/pyfuzzy-toolbox/blob/main/notebooks_colab/Aula_4/01_anfis_iris_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 ANFIS: Classificação de Flores Iris

**Aula 4 - Minicurso de Sistemas de Inferência Fuzzy**

---

## 📋 Objetivo

Neste notebook, vamos aplicar **ANFIS (Adaptive Neuro-Fuzzy Inference System)** para classificar flores Iris.

### O que é ANFIS?

ANFIS combina:
- 🧩 **Lógica Fuzzy**: Regras interpretáveis
- 🧠 **Redes Neurais**: Aprendizado automático (backpropagation)

### Vantagens sobre Wang-Mendel:
- ✅ **Refina** as funções de pertinência (MFs)
- ✅ **Otimiza** parâmetros (não apenas gera regras)
- ✅ **Adapta-se** aos dados via gradient descent

---

## 📚 Referências
- Jang, J. S. (1993). "ANFIS: adaptive-network-based fuzzy inference system." *IEEE transactions on systems, man, and cybernetics*, 23(3), 665-685.
- Fisher, R. A. (1936). "The use of multiple measurements in taxonomic problems."

---

## 🔧 Instalação

In [ ]:
!pip install pyfuzzy-toolbox[ml] scikit-learn -q

print('✅ pyfuzzy-toolbox e scikit-learn instalados com sucesso!')

## 📦 Importações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fuzzy_systems as fs
from fuzzy_systems.learning import ANFIS
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd

%matplotlib inline

# Configurações
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette('husl')
np.random.seed(42)

print('✅ Bibliotecas importadas!')
print(f'   pyfuzzy-toolbox: {fs.__version__}')

## 📊 Passo 1: Carregar e Explorar Dataset Iris

In [ ]:
# Carregar dataset
iris = load_iris()
X_full = iris.data
y_full = iris.target

# Criar DataFrame para facilitar visualização
df = pd.DataFrame(X_full, columns=iris.feature_names)
df['species'] = iris.target_names[y_full]

print('📊 Dataset Iris carregado!')
print(f'\nShape: {X_full.shape}')
print(f'\nClasses: {iris.target_names}')
print(f'\nFeatures: {iris.feature_names}')
print('\nPrimeiras 5 amostras:')
df.head()

## 📈 Visualizar Distribuição das Features

In [ ]:
# Visualizar distribuição das features
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, feature in enumerate(iris.feature_names):
    for species_id, species_name in enumerate(iris.target_names):
        data = df[df['species'] == species_name][feature]
        axes[idx].hist(data, alpha=0.6, bins=15, label=species_name)
    
    axes[idx].set_xlabel(feature, fontsize=11)
    axes[idx].set_ylabel('Frequência', fontsize=11)
    axes[idx].set_title(f'Distribuição: {feature}', fontsize=12, weight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Visualização criada!')

## 🎯 Passo 2: Preparar Dados - Classificação Binária (2 Features)

Vamos começar com um problema **mais simples**:
- Usar apenas **2 features**: Petal Length e Petal Width
- **Classificação binária**: Setosa vs Não-Setosa

**Por quê?** Setosa é linearmente separável das outras espécies!

In [ ]:
# Selecionar apenas as features da pétala (colunas 2 e 3)
X_2d = X_full[:, 2:4]  # petal length e petal width

# Criar labels binários: Setosa (0) vs Outras (1)
y_binary = (y_full != 0).astype(int)  # 0 = Setosa, 1 = Não-Setosa

print('Features selecionadas:')
print(f'  - {iris.feature_names[2]}')
print(f'  - {iris.feature_names[3]}')
print(f'\nDistribuição das classes:')
print(f'  Setosa:     {np.sum(y_binary == 0)} amostras')
print(f'  Não-Setosa: {np.sum(y_binary == 1)} amostras')

## 📈 Visualizar Separabilidade

In [ ]:
# Visualizar separabilidade
fig, ax = plt.subplots(figsize=(10, 8))

colors = ['red', 'blue']
labels = ['Setosa', 'Não-Setosa']

for class_id in [0, 1]:
    mask = y_binary == class_id
    ax.scatter(X_2d[mask, 0], X_2d[mask, 1], 
              c=colors[class_id], label=labels[class_id],
              s=80, alpha=0.7, edgecolors='black', linewidth=1)

ax.set_xlabel(iris.feature_names[2] + ' (cm)', fontsize=13)
ax.set_ylabel(iris.feature_names[3] + ' (cm)', fontsize=13)
ax.set_title('Iris Dataset - 2 Features (Pétala)', fontsize=15, weight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('Note: Setosa é claramente separável das outras classes!')

## 📦 Passo 3: Dividir e Normalizar Dados

In [ ]:
# Dividir dados: 70% treino, 15% validação, 15% teste
X_train_2d, X_temp_2d, y_train_2d, y_temp_2d = train_test_split(
    X_2d, y_binary, test_size=0.3, random_state=42, stratify=y_binary
)

X_val_2d, X_test_2d, y_val_2d, y_test_2d = train_test_split(
    X_temp_2d, y_temp_2d, test_size=0.5, random_state=42, stratify=y_temp_2d
)

print(f'Conjunto de treino:    {len(X_train_2d)} amostras')
print(f'Conjunto de validação: {len(X_val_2d)} amostras')
print(f'Conjunto de teste:     {len(X_test_2d)} amostras')

# Normalizar dados
scaler_2d = StandardScaler()
X_train_2d_norm = scaler_2d.fit_transform(X_train_2d)
X_val_2d_norm = scaler_2d.transform(X_val_2d)
X_test_2d_norm = scaler_2d.transform(X_test_2d)

print('\n✅ Dados normalizados!')
print(f'\nRanges após normalização:')
print(f'  Feature 1: [{X_train_2d_norm[:, 0].min():.2f}, {X_train_2d_norm[:, 0].max():.2f}]')
print(f'  Feature 2: [{X_train_2d_norm[:, 1].min():.2f}, {X_train_2d_norm[:, 1].max():.2f}]')

## 🧠 Passo 4: Criar e Treinar Modelo ANFIS

### Arquitetura ANFIS:

- **2 entradas** (petal_length, petal_width)
- **3 MFs** por entrada → 3² = **9 regras**
- **Regularização L2** para evitar overfitting
- **Learning rate**: 1e-3

In [ ]:
# Criar modelo ANFIS
anfis_2d = ANFIS(
    n_inputs=2,
    n_mfs=3,
    learning_rate=1e-3,
    lambda_l2=0.01,  # Regularização L2
    classification=True
)

print('✅ Modelo ANFIS criado!')
print(f'\n📊 Arquitetura:')
print(f'   • Entradas: 2')
print(f'   • MFs por entrada: 3')
print(f'   • Total de regras: 9')
print(f'   • Learning rate: 0.001')
print(f'   • Regularização: L2 (λ=0.01)')

## 🏋️ Treinar ANFIS

In [ ]:
# Treinar
print('Iniciando treinamento...\n')

anfis_2d.fit(
    X_train_2d_norm, y_train_2d,
    epochs=200,
    validation_data=(X_val_2d_norm, y_val_2d),
    early_stopping_patience=30,
    verbose=True
)

print('\n' + '='*60)
print('✅ Treinamento concluído!')
print('='*60)

## 📈 Visualizar Convergência

In [ ]:
# Plotar curvas de aprendizado
history = anfis_2d.history

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss
ax1.plot(history['loss'], 'b-', linewidth=2, label='Treino', alpha=0.8)
ax1.plot(history['val_loss'], 'r--', linewidth=2, label='Validação', alpha=0.8)
ax1.set_xlabel('Época', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Convergência - Loss', fontsize=14, weight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Accuracy
ax2.plot(history['accuracy'], 'b-', linewidth=2, label='Treino', alpha=0.8)
ax2.plot(history['val_accuracy'], 'r--', linewidth=2, label='Validação', alpha=0.8)
ax2.set_xlabel('Época', fontsize=12)
ax2.set_ylabel('Acurácia', fontsize=12)
ax2.set_title('Convergência - Acurácia', fontsize=14, weight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Curvas de aprendizado geradas!')

## 🎯 Passo 5: Avaliar Desempenho

In [ ]:
# Predições
y_train_pred_2d = anfis_2d.predict(X_train_2d_norm)
y_val_pred_2d = anfis_2d.predict(X_val_2d_norm)
y_test_pred_2d = anfis_2d.predict(X_test_2d_norm)

# Acurácias
acc_train_2d = accuracy_score(y_train_2d, y_train_pred_2d)
acc_val_2d = accuracy_score(y_val_2d, y_val_pred_2d)
acc_test_2d = accuracy_score(y_test_2d, y_test_pred_2d)

print('='*70)
print('📊 RESULTADOS - CLASSIFICAÇÃO BINÁRIA (2 FEATURES)')
print('='*70)
print(f'Acurácia Treino:    {acc_train_2d*100:6.2f}%')
print(f'Acurácia Validação: {acc_val_2d*100:6.2f}%')
print(f'Acurácia Teste:     {acc_test_2d*100:6.2f}%')
print('='*70)

## 📊 Matriz de Confusão

In [ ]:
# Matriz de confusão
cm_test_2d = confusion_matrix(y_test_2d, y_test_pred_2d)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_test_2d, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Setosa', 'Não-Setosa'],
            yticklabels=['Setosa', 'Não-Setosa'],
            cbar_kws={'label': 'Contagem'})
ax.set_xlabel('Predito', fontsize=13)
ax.set_ylabel('Real', fontsize=13)
ax.set_title(f'Matriz de Confusão - Teste\nAcurácia: {acc_test_2d*100:.2f}%',
            fontsize=14, weight='bold')
plt.tight_layout()
plt.show()

# Relatório detalhado
print('\n📋 RELATÓRIO DE CLASSIFICAÇÃO - CONJUNTO DE TESTE')
print('='*70)
print(classification_report(y_test_2d, y_test_pred_2d,
                           target_names=['Setosa', 'Não-Setosa'],
                           digits=4))

## 🗺️ Visualizar Fronteira de Decisão

In [ ]:
# Criar grid para a superfície de decisão
x1_min, x1_max = X_train_2d_norm[:, 0].min() - 0.5, X_train_2d_norm[:, 0].max() + 0.5
x2_min, x2_max = X_train_2d_norm[:, 1].min() - 0.5, X_train_2d_norm[:, 1].max() + 0.5

xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 200),
                       np.linspace(x2_min, x2_max, 200))

# Fazer predições no grid
print('Gerando superfície de decisão...')
Z = anfis_2d.predict(np.c_[xx1.ravel(), xx2.ravel()]).reshape(xx1.shape)
print('Superfície gerada!')

# Plotar fronteira de decisão
fig, ax = plt.subplots(figsize=(12, 10))

# Superfície
contour = ax.contourf(xx1, xx2, Z, levels=[0, 0.5, 1],
                      colors=['#ffcccc', '#ccccff'], alpha=0.6)

# Contorno da fronteira
ax.contour(xx1, xx2, Z, levels=[0.5], colors='black',
           linewidths=3, linestyles='solid')

# Pontos com indicação de erro
corretos = y_test_2d == y_test_pred_2d
incorretos = ~corretos

ax.scatter(X_test_2d_norm[corretos, 0], X_test_2d_norm[corretos, 1],
           c='green', marker='o', s=100, label='Correto',
           edgecolors='black', linewidth=1.5, alpha=0.8)
ax.scatter(X_test_2d_norm[incorretos, 0], X_test_2d_norm[incorretos, 1],
           c='orange', marker='X', s=150, label='Erro',
           edgecolors='red', linewidth=2, alpha=0.9)

ax.set_xlabel('Comprimento da Pétala (normalizado)', fontsize=12)
ax.set_ylabel('Largura da Pétala (normalizado)', fontsize=12)
ax.set_title(f'Fronteira de Decisão ANFIS\nErros: {np.sum(incorretos)}/{len(y_test_2d)}',
             fontsize=14, weight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 🎓 Conclusões

### ✅ O que aprendemos?

1. **ANFIS combina o melhor de dois mundos**:
   - **Fuzzy**: Regras interpretáveis
   - **Neural**: Aprendizado automático via backpropagation

2. **Vantagens sobre Wang-Mendel**:
   - Wang-Mendel: Gera regras, MFs fixas
   - ANFIS: **Refina** MFs e consequentes via gradient descent

3. **Regularização é importante**:
   - L2 (Ridge) evita overfitting
   - Especialmente útil com poucas amostras

4. **Trade-offs**:
   - ANFIS: Mais parâmetros, mais flexível, treino mais lento
   - Wang-Mendel: Menos parâmetros, mais rápido, menos flexível

---

### 🔬 ANFIS vs Wang-Mendel

| Aspecto | Wang-Mendel | ANFIS |
|---------|-------------|-------|
| Geração de regras | Automática (1 passo) | Inicialização + refinamento |
| Funções de pertinência | Fixas | **Aprendidas** |
| Consequentes | Fixos | **Otimizados** |
| Tempo de treino | Rápido | Mais lento |
| Flexibilidade | Menor | **Maior** |
| Interpretabilidade | Alta | Alta (se MFs não mudam muito) |

---

### 🚀 Próximos Passos

1. **Experimentar com 4 features** (todas as variáveis do Iris)
2. **Testar diferentes valores de regularização** (λ)
3. **Comparar com outros métodos** (SVM, Random Forest)
4. **Aplicar a outros datasets** de classificação

---

### 📚 Recursos

- **PyPI**: https://pypi.org/project/pyfuzzy-toolbox/
- **GitHub**: https://github.com/1moi6/pyfuzzy-toolbox

### 📖 Referências

1. **Jang, J. S. (1993)**. "ANFIS: adaptive-network-based fuzzy inference system." *IEEE transactions on systems, man, and cybernetics*, 23(3), 665-685.

2. **Fisher, R. A. (1936)**. "The use of multiple measurements in taxonomic problems." *Annals of Eugenics*, 7(2), 179-188.

---

**🎉 Parabéns! Você aplicou ANFIS para classificação!**

*Notebook desenvolvido para o Minicurso de Sistemas de Inferência Fuzzy - Aula 4 - 2025*  
*Usando pyfuzzy-toolbox v1.0.0*